# X-collapsed CNN




In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import confusion_matrix

2024-09-22 17:10:58.968069: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-22 17:10:58.996068: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 17:10:58.996086: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 17:10:58.997167: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 17:10:59.003251: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# List all physical devices of type 'GPU'
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    print(f'Number of GPUs available: {len(gpus)}')
    for i, gpu in enumerate(gpus):
        print(f'GPU {i}: {gpu}')
else:
    print('No GPU detected.')

Number of GPUs available: 1
GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
# full list of all schizophrenia IDs from csv, 86 in total but not all satisfy have t>90

full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [4]:
def build_3d_cnn(input_shape, num_classes=1):
    model = Sequential([
        Conv3D(64, kernel_size=(10, 10, 10), activation='relu', input_shape=input_shape),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Conv3D(128, kernel_size=(5, 5, 5), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Conv3D(128, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Flatten(),

        Dense(5000, activation='relu'),
        Dropout(0.3),  
        Dense(5000, activation='relu'),
        Dropout(0.3),
        Dense(5000, activation='relu'),
        Dropout(0.3),

        Dense(num_classes, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


input_shape = (91, 91, 146, 1)
model = build_3d_cnn(input_shape)
model.summary()

2024-09-22 17:11:00.340075: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46748 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:17:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 82, 82, 137, 64)   64064     
                                                                 
 max_pooling3d (MaxPooling3  (None, 41, 41, 68, 64)    0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 41, 41, 68, 64)    256       
 Normalization)                                                  
                                                                 
 conv3d_1 (Conv3D)           (None, 37, 37, 64, 128)   1024128   
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 18, 18, 32, 128)   0         
 g3D)                                                            
                                                        

In [5]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0]/image.shape[0],
        new_shape[1]/image.shape[1],
        new_shape[2]/image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation

In [6]:
total_accuracy = 0.0
total_loss = 0.0
num_iterations =1

for i in range(num_iterations):

    ''' data training for GAN '''
    ''' Choosing 50 random schizphrenia samples and store remaining 9 test'''
    ''' Since we are training a separate GAN for control, also choose 50 random control samples and store remaining 9 test '''

    # GAN Training Data Selection
    gan_train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 50)
    gan_test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in gan_train_ids_schiz]

    gan_train_ids_control = random.sample(met_requirement_control_ids, 50)
    gan_test_ids_control = [id for id in met_requirement_control_ids if id not in gan_train_ids_control]

    # since we have 82 - 50 = 32, choose 9 random for testing    
    gan_test_ids_control = random.sample(gan_test_ids_control,9)

    ''' data training for classifier '''
    ''' just use the same train set as GAN above '''

    # Classifier Training Data Selection
    classifier_train_ids = gan_train_ids_control + gan_train_ids_schiz

    # Classifier Test Data Selection
    classifier_test_ids = gan_test_ids_schiz + gan_test_ids_control

    ''' File loading '''
    # Specify the directory and file pattern
    directory_path = '../4D/'
    file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

    # Construct the full path pattern
    path_pattern = f'{directory_path}/{file_pattern}'

    # Use glob to find all matching files
    matching_files = glob.glob(path_pattern)

    ''' File loading for GAN Training and classifer '''
    ''' But this time we have 2 separate GANs, 1 train on schizoprenia and 1 train on control'''
    
    classifier_image_data = []
    classifier_labels = []  # 1 for schizophrenia, 0 for non-schizophrenia
    gan_image_data_schiz = []
    gan_image_data_control = []

    for file_path in matching_files:
        filename = os.path.basename(file_path)
        file_id = filename.split('_')[0]
        
        if file_id in gan_train_ids_schiz:
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()

            if t1_data.shape[3] < 90:
                continue

            t1_data_collapsed = np.sum(t1_data, axis=0) # 0 for x axis
            gan_image_data_schiz.append(t1_data_collapsed)

        if file_id in gan_train_ids_control:
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()

            if t1_data.shape[3] < 90:
                continue

            t1_data_collapsed = np.sum(t1_data, axis=0)
            gan_image_data_control.append(t1_data_collapsed)

        if file_id in classifier_train_ids or file_id in classifier_test_ids:
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()

            if t1_data.shape[3] < 90:
                continue

            label = 1 if file_id in met_requirement_schizophrenia_ids else 0

            # collapsed x-axis
            t1_data_collapsed = np.sum(t1_data, axis=0)
            classifier_image_data.append(t1_data_collapsed)
            classifier_labels.append(label)

    print(f"Total GAN training files processed: {len(gan_image_data_control+gan_image_data_schiz)}")
    print(f"Total classifier training/testing files processed: {len(classifier_image_data)}")
    print(f"Total labels processed: {len(classifier_labels)}")
  

    '''Determine the maximum t-dimension size '''
    max_z_size_schiz = max(img.shape[2] for img in gan_image_data_schiz)
    max_z_size_control = max(img.shape[2] for img in gan_image_data_control)
    

    ''' normalization '''
    image_data_normalized_schiz = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_schiz]
    image_data_normalized_control = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_control]
    

    ''' padding of images data '''
    # Pad each image to have a consistent z-dimension size
    padded_data_schiz = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size_schiz - img.shape[2])), mode='constant') for img in image_data_normalized_schiz]
    padded_data_control = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size_control - img.shape[2])), mode='constant') for img in image_data_normalized_control]

    target_size = (91, 91, 146)

    # Apply resizing to GAN training images
    resized_data_schiz = [resize_image(img, target_size) for img in padded_data_schiz]
    resized_data_control = [resize_image(img, target_size) for img in padded_data_control]

    
    
    # Now convert the padded data list to a numpy array
    padded_data_array_schiz = np.array(resized_data_schiz)
    padded_data_array_control = np.array(resized_data_control)

    ''' loading the data for WGAN training '''
    train_images_schiz = padded_data_array_schiz
    train_images_control = padded_data_array_control

    # Define batch size
    batch_size = 10

    train_dataset_schiz = tf.data.Dataset.from_tensor_slices((train_images_schiz)).shuffle(len(train_images_schiz)).batch(batch_size)
    train_dataset_control = tf.data.Dataset.from_tensor_slices((train_images_control)).shuffle(len(train_images_control)).batch(batch_size)
    
    # Create labels for the datasets
    labels_schiz = np.ones(len(padded_data_array_schiz))  # 1 for schizophrenia
    labels_control = np.zeros(len(padded_data_array_control))  # 0 for control

    # Combine the data and labels
    combined_images = np.concatenate([padded_data_array_schiz, padded_data_array_control], axis=0)
    combined_labels = np.concatenate([labels_schiz, labels_control], axis=0)

    # Shuffle the combined dataset to ensure the data is randomly distributed
    # It's important for training a model that generalizes well
    indices = np.arange(combined_images.shape[0])
    np.random.shuffle(indices)
    shuffled_images = combined_images[indices]
    shuffled_labels = combined_labels[indices]

    # Convert to TensorFlow Dataset, manually split 80/20
    dataset = tf.data.Dataset.from_tensor_slices((shuffled_images, shuffled_labels))
    train_size = int(0.8 * len(shuffled_images))
    train_dataset = dataset.take(train_size).shuffle(buffer_size=train_size).batch(10)
    test_dataset = dataset.skip(train_size).batch(10)

    # Train the model
    history = model.fit(
        train_dataset,
        epochs=100,
        validation_data=test_dataset
    )
    # Evaluate the model
    train_loss, train_acc = model.evaluate(test_dataset)
    print(f"Training accuracy: {train_acc}, Training loss: {train_loss}")


    ''' load test images to make test set'''
    # Now proceed with loading and preprocessing the images for these IDs
    test_image_data = []
    test_labels = []

    test_ids = classifier_test_ids

    # Loop through the matching files and filter based on test IDs
    for file_path in matching_files:
        filename = os.path.basename(file_path)
        file_id = filename.split('_')[0]

        # Process only if the ID is in the test set
        if file_id in test_ids:
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()

            # Ensure sufficient time dimension
            if t1_data.shape[3] < 90:
                continue

            # Collapse one of the axes by summing
            t1_data_collapsed = np.sum(t1_data, axis=0)

            # Resize, normalize, and add dimension as done in the training data preparation
            processed_image = resize_image(t1_data_collapsed, (91, 91, 146))
            processed_image_normalized = (processed_image - np.min(processed_image)) / (np.max(processed_image) - np.min(processed_image)) * 2 - 1
            processed_image_final = np.expand_dims(processed_image_normalized, axis=-1)

            test_image_data.append(processed_image_final)
            label = 1 if file_id in met_requirement_schizophrenia_ids else 0
            test_labels.append(label)

    # Convert to numpy arrays
    test_images_array = np.array(test_image_data)
    test_labels_array = np.array(test_labels)


    test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)

    loss, accuracy = model.evaluate(test_dataset)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)

    
    total_loss += loss
    total_accuracy += accuracy
    
    predicted_probs = model.predict(test_dataset)
    predicted_labels = np.where(predicted_probs > 0.5, 1, 0)  # Convert probabilities to binary labels

    # Extract the actual labels from the test dataset
    actual_labels = np.concatenate([y for x, y in test_dataset], axis=0)

    # Print the predicted labels vs actual labels
    print("Predicted Labels vs Actual Labels:")
    for pred, actual in zip(predicted_labels, actual_labels):
        print(f"Predicted: {pred}, Actual: {actual}")

    # Generate a confusion matrix
    cm = confusion_matrix(actual_labels, predicted_labels)
    print("\nConfusion Matrix:")
    print(cm)

# Calculate and print the average loss and accuracy over all iterations
average_test_loss = total_loss / num_iterations
average_test_accuracy = total_accuracy / num_iterations
print(f"\nAverage Test Loss over {num_iterations} iterations: {average_test_loss}")
print(f"Average Test Accuracy over {num_iterations} iterations: {average_test_accuracy}")

Total GAN training files processed: 100
Total classifier training/testing files processed: 118
Total labels processed: 118
Epoch 1/100


2024-09-22 17:17:23.082510: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-09-22 17:17:43.040497: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f47c92a5fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-22 17:17:43.040522: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-09-22 17:17:43.044680: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1727039863.118540 2282886 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8/8 [==============================] - 54s 3s/step - loss: 135.5986 - accuracy: 0.5500 - val_loss: 2103.2065 - val_accuracy: 0.5500
Epoch 2/100
8/8 [==============================] - 4s 544ms/step - loss: 201.3901 - accuracy: 0.6125 - val_loss: 163.5262 - val_accuracy: 0.5000
Epoch 3/100
8/8 [==============================] - 4s 545ms/step - loss: 75.3495 - accuracy: 0.5375 - val_loss: 1123.4763 - val_accuracy: 0.5500
Epoch 4/100
8/8 [==============================] - 4s 546ms/step - loss: 65.4640 - accuracy: 0.4250 - val_loss: 56.1162 - val_accuracy: 0.5000
Epoch 5/100
8/8 [==============================] - 4s 547ms/step - loss: 26.7640 - accuracy: 0.5375 - val_loss: 279.4400 - val_accuracy: 0.5000
Epoch 6/100
8/8 [==============================] - 4s 549ms/step - loss: 28.3859 - accuracy: 0.5750 - val_loss: 112.3668 - val_accuracy: 0.4500
Epoch 7/100
8/8 [==============================] - 4s 550ms/step - loss: 37.2756 - accuracy: 0.6000 - val_loss: 16.6437 - val_accuracy: 0.6500
Epoc